## Домашнее задание 2

In [1]:
import pickle
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from collections import Counter
from scipy.stats import spearmanr, kendalltau
from liblinear import liblinearutil

### 1. Загрузим данные и выделим тренировочную и тестовую выборку

In [2]:
with open('players.pkl', 'rb') as f:
    players = pickle.load(f)
with open('tournaments.pkl', 'rb') as f:
    tournaments = pickle.load(f)
with open('results.pkl', 'rb') as f:
    results = pickle.load(f)

In [3]:
def get_train(results, tournaments):
    new_results = dict()
    new_tournaments = dict()
    for i in results:
        if tournaments[i]['dateStart'][:4] != '2019':          # берем ткрниры за 2019 год
            continue
        res = results[i]
        new_res = []
        mask_len = 0
        for team in res:                                       # для каждого турнира находим наибольшую длину маски
            if ('mask' in team) and (team['mask'] is not None):
                mask_len = max(mask_len, len(team['mask']))
        for team in res:
            if len(team['teamMembers']) == 0:                  # выбрасываем команды без списка участников
                continue
            if ('mask' not in team) or (team['mask'] is None): # выбрасываем без маски ответов
                continue
            if len(team['mask']) != mask_len:                  # выбрасываем команды с другой длиной маски
                continue
            new_res.append(team)       

        if len(new_res) > 0:
            new_results[i] = new_res
            new_tournaments[i] = tournaments[i]
    print("Train size: {}".format(len(new_results)))
    return new_results, new_tournaments

In [4]:
def get_test(results, tournaments):
    new_results = dict()
    new_tournaments = dict()
    for i in results:
        if tournaments[i]['dateStart'][:4] != '2020':                   # берем ткрниры за 2020 год
            continue
        res = results[i]
        new_res = []
        mask_len = 0
        for team in res:
            if len(team['teamMembers']) == 0:                           # выбрасываем команды без позиции
                continue
            if ('position' not in team) or (team['position'] is None):  # выбрасываем без списка участников
                continue
            new_res.append(team)

        if len(new_res) > 0:
            new_results[i] = new_res
            new_tournaments[i] = tournaments[i]
    print("Test size: {}".format(len(new_results)))
    return new_results, new_tournaments

In [5]:
def unique_players(train_results, min_games):
    players_ids = Counter()
    for i in train_results:                            # находим количество сыгранных вопросов для каждого игрока
        res = train_results[i]
        for team in res:
            members = team['teamMembers']
            for player in members:
                id = player['player']['id']
                players_ids[id] += len(team['mask'])
    
        players_ids_often = list()
    for p in players_ids:
        if players_ids[p] > 40:                        # берем у кого более 40 вопросов (иначе памяти не хватает)
            players_ids_often.append(p)
            
    players_dict = dict()
    for i, idx in enumerate(players_ids_often):        # словарь: id -> индекс в списке
        players_dict[idx] = i
    print("Уникальных игроков с более {} игр: {}".format(min_games, len(players_dict)))
    return players_dict, players_ids

In [6]:
def quest_number(train_results):
    counter = 0
    quest_dict = dict()
    for i in train_results:
        res = train_results[i]
        mask_len = len(res[0]['mask'])
        for j in range(mask_len):
            quest_dict[counter] = i        # словарь: вопрос -> турнир
            counter += 1                   # количество вопросов
    print("Вопросов: {}".format(counter))
    return counter, quest_dict

In [7]:
train_results, train_tournaments = get_train(results, tournaments)
test_results, test_tournaments = get_test(results, tournaments)

Train size: 675
Test size: 176


In [8]:
counter, quest_dict = quest_number(train_results)
players_dict, players_quest_number = unique_players(train_results, 2)

Вопросов: 33385
Уникальных игроков с более 2 игр: 39383


### 2. Обучим baseline модель (LogReg)

- Строки - вопросы
- Столбцы - игроки и вопросы
- Если игрок не играл вопрос 0, если ответил 1, не ответил -1
- И 1 в столбце соответсвующем вопросу (диагональ единиц)

In [9]:
X_train = np.zeros((counter, len(players_dict) + counter), dtype='float16')
y_train = np.ones(counter, dtype='float32')

In [10]:
def get_X(X, train_results, players_dict):
    counter = 0
    y_train = []
    for i in train_results:
        res = train_results[i]
        mask_len = len(res[0]['mask'])
        for j in range(mask_len):
            for team in res:
                answer = 1 if team['mask'][j] == '1' else -1
                for player in team['teamMembers']:
                    player_id = player['player']['id']
                    if player_id in players_dict:
                        player_w = players_dict[player_id]
                        X[counter, player_w] = answer
            X[counter, len(players_dict) + counter] = 1
            counter += 1

In [11]:
def train_base(X, y):
    prob = liblinearutil.problem(y_train, X_train)
    param = liblinearutil.parameter('-s 0 -c 0.001')
    liblin = liblinearutil.train(prob, param)
    [W, b] = liblin.get_decfun()
    
    # Силой игрока будем считать вес соответсвующего признака
    # Сдвинем веса так, что наименьшая сила - 0
    W_copy = W.copy()
    min_w = min(W_copy)
    W_pos = [x - min_w for x in W_copy]
    return W_pos

In [12]:
get_X(X_train, train_results, players_dict)
W = train_base(X_train, y_train)

### 3. Предсказание позиций на турнирах 2020

In [13]:
# В качестве силы команды берем взвешенную силу трех лучших игроков (подбирал руками)
def get_team_power(power):
    power = sorted(power, reverse=True)[:3]
    power = power + [0] * (3-len(power))
    if len(power) == 0:
        power = 0
    else:
        power = power[0] * 0.7 + power[1] * 0.2 + power[2] * 0.1
    return power

In [14]:
def corr_metric(W):
    corr_spear = []
    corr_kend = []
    for i in test_results:
        res = test_results[i]
        real = []
        pred = []
        for team in res:
            real.append(team['position'])
            power = []
            for player in team['teamMembers']:
                id = player['player']['id']
                if id in players_dict:
                    w_idx = players_dict[id]
                    power.append(W[w_idx])
            power = get_team_power(power)
            pred.append(power)
        real.reverse()
        corr_spear.append(spearmanr(real, pred)[0])
        corr_kend.append(kendalltau(real, pred)[0])
    corr_spear = [x for x in corr_spear if str(x) != 'nan']
    corr_kend = [x for x in corr_kend if str(x) != 'nan']
    corr_spear = round(sum(corr_spear) / len(corr_spear), 4)
    corr_kend = round(sum(corr_kend) / len(corr_kend), 4)
    print("Spearman: {}".format(corr_spear))
    print("Kendall: {}".format(corr_kend))
    return corr_spear

In [15]:
corr_metric(W);

/usr/local/lib/python3.5/dist-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/usr/local/lib/python3.5/dist-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/usr/local/lib/python3.5/dist-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/usr/local/lib/python3.5/dist-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/usr/local/lib/python3.5/dist-packages/scipy/stats/_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


Spearman: 0.6038
Kendall: 0.4548


### 5. Топ вопросов по сложности

In [16]:
# Большая часть вопросов в топе с чемпионатов мира и России, что выглядит логично
W_ans = W[len(players_dict):]
W_ans = [(x, i) for i, x in enumerate(W_ans)]
W_ans = sorted(W_ans, key=lambda x: x[0], reverse=True)
for i in range(20):
    idx_tour = quest_dict[W_ans[i][1]]
    print(tournaments[idx_tour]['name'])

Синхрон-lite. Выпуск XXII
Кубок городов
Пущинские Дали - синхрон
Чемпионат Мира. Этап 2. Группа А
Чемпионат России
Чемпионат Мира. Финал. Группа А
Чемпионат Мира. Этап 2. Группа А
KFC
Чемпионат России
Чемпионат России
Чемпионат России
Veidrodis
Чемпионат Мира. Этап 3. Группа А
Молодёжный чемпионат Нижегородской области
Чемпионат России
Чемпионат России
Чемпионат Мира. Финал. Группа А
Чемпионат Мира. Финал. Группа А
Чемпионат России
Донат


### 6. Рейтинг игроков

Рейтинг игроков составляю по результатам baseline модели, так как, не смотря на улучшение метрик при использовании EM-алгоритма, рейтинг становится менее правдоподбным

In [17]:
def get_players_scor(W):
    players_scor = {
        'scor': [],
        'questions_number': [],
        'name': []
    }
    for i in players:
        player = players[i]
        id = player['id']
        if id in players_dict:
            idx = players_dict[id]
            players_scor['scor'].append(W[idx])
            players_scor['questions_number'].append(players_quest_number[id])
            players_scor['name'].append('{} {} {}'.format(player['surname'], player['name'], player['patronymic']))
    players_scor = pd.DataFrame(players_scor).sort_values('scor', ascending=False)
    return players_scor

Часть ироков из топа получившегося рейтинга действительно занимают верхние строчки, но видна корреляция с количеством сыгранных вопросов, которая, видимо, и не позволила достичь лучшего качества модели и побороть ее не удалось. При чем это противоречит предположению о новичках в топе, и убирать их специально не требуется

In [18]:
W_players = W[:len(players_dict)]
players_scor = get_players_scor(W_players)
players_scor[['name', 'questions_number']].head(20)

,name,questions_number
10554,Дворянчиков Алексей Ярославович,8575
3917,Спешков Сергей Леонидович,3767
3897,Сорожкин Артём Сергеевич,4885
3718,Семушин Иван Николаевич,3803
3817,Слинявчук Виктория Викторовна,5487
1595,Ильин Константин Евгеньевич,8139
2389,Либер Александр Витальевич,3821
17371,Мощенков Витольд Валентинович,1830
9103,Комар Наталья Александровна,4340
4681,Штых Алексей Константинович,3158


### 4. EM-алгоритм

Предположим, что вклад в ответ команды пропорционален силе игрока. Тогда совокупность этих значений по всем игрокам и будут скрытыми переменными. Для удучшения качества модели, немного меняем схему
- Сила команды равна просто среднему значению по команде
- В случае неответа на вопрос ставим 0, а не -1
- Если игрок ответил на вопрос, ставим не 1, а силу игрока (пробовал нормировать на сумму сил в команде - не помогло)

In [19]:
def e_step(X, y):
    prob = liblinearutil.problem(y, X)
    param = liblinearutil.parameter('-s 0 -c 1 -B 1')
    liblin = liblinearutil.train(prob, param)
    [W, b] = liblin.get_decfun()
    return W

In [20]:
def m_step(X, W):
    counter = 0
    y_train = []
    for i in train_results:
        res = train_results[i]
        mask_len = len(res[0]['mask'])
        for j in range(mask_len):
            for team in res:
                team_power = 0
                for player in team['teamMembers']:
                    player_id = player['player']['id']
                    if player_id in players_dict:
                        player_w = players_dict[player_id]
                        player_power = W[player_w]
                        team_power += player_power
                answer = 1 if team['mask'][j] == '1' else 0
                for player in team['teamMembers']:
                    player_id = player['player']['id']
                    if player_id in players_dict:
                        player_w = players_dict[player_id]
                        player_power = W[player_w] + 0.001
                        X[counter, player_w] = answer * player_power
            counter += 1

In [21]:
def get_team_power(power):
    if len(power) == 0:
        return 0
    return sum(power) / (len(power))

In [22]:
X_train = np.zeros((counter, len(players_dict)), dtype='float16')
y_train = np.ones(counter, dtype='float16')

In [24]:
W = [1] * (len(players_dict))
best_spearmen = 0
best_w = []
for i in range(10):
    print(i)
    m_step(X_train, W)           # обновляем матрицу в соотвествии с силой игроков
    W = e_step(X_train, y_train) # обновляем силу
    spearmen = corr_metric(W)
    if spearmen > best_spearmen:
        best_spearmen = spearmen
        best_w = W

0


/usr/local/lib/python3.5/dist-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/usr/local/lib/python3.5/dist-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/usr/local/lib/python3.5/dist-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/usr/local/lib/python3.5/dist-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/usr/local/lib/python3.5/dist-packages/scipy/stats/_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


Spearman: 0.5697
Kendall: 0.4252
1
Spearman: 0.6234
Kendall: 0.4708
2
Spearman: 0.6299
Kendall: 0.4775
3
Spearman: 0.6287
Kendall: 0.4769
4
Spearman: 0.6285
Kendall: 0.4769
5
Spearman: 0.6298
Kendall: 0.4775
6
Spearman: 0.6318
Kendall: 0.4789
7
Spearman: 0.6322
Kendall: 0.4792
8
Spearman: 0.6322
Kendall: 0.4792
9
Spearman: 0.6302
Kendall: 0.4776


In [25]:
print('Best Spearmen: {}'.format(best_spearmen))

Best Spearmen: 0.6322


### 7. Глубина рейтинга

- Из-за объема данных обучиться на всех турнирах не удалось
- Идея: Продублировать строки, соответвующие вопросам предпоследнего года, и взять 4 раза вопросы последнего года. Так мы увеличим веса этих объектов (вопросов) для модели